In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
import pandas as pd
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import logging
logger = logging.getLogger(__name__)
# Dataset personalizado que carga CSV desde la ruta local del nodo
class BreastCancerDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        # Asumiendo que las columnas de características están en df.columns[:-1]
        # y la última columna es la etiqueta
        X = df.iloc[:, :-1].values.astype(np.float32)
        y = df.iloc[:, -1].values.astype(np.int64)

        # Normalizar características
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        self.X = torch.tensor(X)
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

from fedbiomed.common.training_plans import TorchTrainingPlan
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np

class MyTrainingPlan(TorchTrainingPlan):

    class BreastCancerDataset(torch.utils.data.Dataset):
        def __init__(self, csv_path):
            df = pd.read_csv(csv_path)
            X = df.iloc[:, :-1].values.astype(np.float32)
            y = df.iloc[:, -1].values.astype(np.int64)
            scaler = StandardScaler()
            X = scaler.fit_transform(X)
            self.X = torch.tensor(X)
            self.y = torch.tensor(y)

        def __len__(self):
            return len(self.y)

        def __getitem__(self, idx):
            return self.X[idx], self.y[idx]

    def init_model(self, model_args):
        class Net(nn.Module):
            def __init__(self):
                super().__init__()
                self.fc1 = nn.Linear(512, 16)
                self.fc2 = nn.Linear(16, 2)

            def forward(self, x):
                x = F.relu(self.fc1(x))
                x = self.fc2(x)
                return F.log_softmax(x, dim=1)
        return Net()

    def init_optimizer(self, optimizer_args):
        return torch.optim.Adam(self.model().parameters(), lr=optimizer_args['lr'])

    def init_dependencies(self):
        return [
            "import numpy as np",
            "import pandas as pd",
            "from sklearn.preprocessing import StandardScaler",
            "import torch",
            "import torch.nn.functional as F",
            "from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score",
            "from sklearn.model_selection import train_test_split",  # <--- esta línea es la clave
            "from torch.utils.data import TensorDataset"
        ]

    #def training_data(self):
     #   dataset = self.BreastCancerDataset(self.dataset_path)
      #  return DataManager(dataset=dataset, batch_size=32, shuffle=True)
    
    def training_data(self):
        df = pd.read_csv(self.dataset_path)
        X = df.iloc[:, :-1].values.astype(np.float32)
        y = df.iloc[:, -1].values.astype(np.int64)
        X = StandardScaler().fit_transform(X)
    
        # Split manual (como Fed-BioMed lo haría)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
        # Guarda el conjunto de test para luego evaluarlo manualmente si deseas
        self.X_val = torch.tensor(X_test)
        self.y_val = torch.tensor(y_test)
    
        dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
        return DataManager(dataset=dataset, batch_size=32, shuffle=True)
        
    def training_step(self, data, target, epoch=None):
        # Entrenamiento normal
        self.model().train()
        output = self.model().forward(data)
        loss = F.nll_loss(output, target)
        return loss
        
   
    def evaluate(self, data, target):
        """
        Fed-BioMed invoca esto sobre cada batch de validación
        (cuando tengas test_on_global_updates=True).
        Debe devolver un dict {<metric_name>: value, …}.
        """
        logger.info(">>> Ejecutando método evaluate personalizado")
        #raise RuntimeError("Este evaluate() sí se está ejecutando.")
        self.model().eval()
        with torch.no_grad():
            logits = self.model()(data)
            loss   = F.nll_loss(logits, target).item()
            preds  = logits.argmax(dim=1).cpu().numpy()
            y_true = target.cpu().numpy()

            # calcula todas tus métricas
            acc  = (preds == y_true).mean()
            prec = precision_score(y_true, preds, average='weighted', zero_division=0)
            rec  = recall_score(y_true, preds, average='weighted', zero_division=0)
            f1   = f1_score(y_true, preds, average='weighted', zero_division=0)

            metrics = {
                        'global/loss':      loss,
                        'global/accuracy':  acc,
                        'global/precision': prec,
                        'global/recall':    rec,
                        'global/f1_score':  f1
            }
    
            # loguea las métricas para TensorBoard
            for name, val in metrics.items():
                self.log(name, val)

            logger.info(f"Evaluating metrics: acc={acc}, prec={prec}, recall={rec}, f1={f1}")

    
        return metrics





In [2]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage
from fedbiomed.common.metrics import MetricTypes

tags = ['breast', 'cancer','mammography']  # los tags con los que etiquetaste tu dataset en los nodos
rounds = 5
model_args = {}

# ——— Cambia aquí: añade test_ratio, test_on_global_updates y test_metric ———
training_args = {
    'loader_args':            {'batch_size': 32},
    'optimizer_args':         {'lr': 1e-3},
    'epochs':                 5,
    'dry_run':                False,
    'batch_maxnum':           100,

    # split local train/val y evalúa globalmente
    'test_ratio':             0.2,  
    'test_on_global_updates': True,
    'test_metric': MetricTypes.F1_SCORE,
    'test_on_global_updates': True,
    'test_on_local_updates': True,
    'log_interval':           10
}

exp = Experiment(
    tags=tags,
    model_args=model_args,
    training_plan_class=MyTrainingPlan,
    training_args=training_args,
    round_limit=rounds,
    aggregator=FedAverage(),
    tensorboard=True
)

# Ejecuta el entrenamiento y guarda el resultado
results = exp.run()
# Guardas los pesos aprendidos en un .pth
exp.training_plan().export_model('modelo_federado_prueba.pth')



2025-08-01 18:31:15,807 fedbiomed INFO - Starting researcher service...

2025-08-01 18:31:15,817 fedbiomed INFO - Waiting 3s for nodes to connect...

2025-08-01 18:31:16,930 fedbiomed DEBUG - Node: NODE_1396bb40-b8fc-4d8f-9575-8a5bba5b6ebb polling for the tasks

2025-08-01 18:31:18,824 fedbiomed INFO - Updating training data. This action will update FederatedDataset, and the nodes that will participate to the experiment.

2025-08-01 18:31:18,891 fedbiomed DEBUG - Node: NODE_1396bb40-b8fc-4d8f-9575-8a5bba5b6ebb polling for the tasks

...

2025-08-01 18:31:27,994 fedbiomed INFO - Nodes that successfully reply in round 4 ['NODE_1396bb40-b8fc-4d8f-9575-8a5bba5b6ebb']

<function extract_symbols at 0x7fa3d0b5fb50>


2025-08-01 18:31:27,997 fedbiomed INFO - Sending request 
					 To: NODE_1396bb40-b8fc-4d8f-9575-8a5bba5b6ebb 
					 Request: :Perform final validation on aggregated parameters 
 -----------------------------------------------------------------

2025-08-01 18:31:28,006 fedbiomed DEBUG - Node: NODE_1396bb40-b8fc-4d8f-9575-8a5bba5b6ebb polling for the tasks

2025-08-01 18:31:28,480 fedbiomed INFO - VALIDATION ON GLOBAL UPDATES 
					 NODE_ID: NODE_1396bb40-b8fc-4d8f-9575-8a5bba5b6ebb 
					 Round 6 | Iteration: 1/1 (100%) | Samples: 1302/1302
 					 F1_SCORE: 1.000000 
					 ---------

In [3]:
try:
    exp.training_plan().export_model('./trained_model')
except Exception as e:
    print(e)

print("Training rounds keys:", exp.training_replies().keys())
print("Aggregated params keys:", exp.aggregated_params().keys())

Training rounds keys: dict_keys([0, 1, 2, 3, 4])
Aggregated params keys: dict_keys([0, 1, 2, 3, 4])


In [1]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 1) Recupera tu TrainingPlan en una variable
plan = exp.training_plan()
# 2) Importa los pesos que ya exportaste
plan.import_model('modelo_federado_prueba.pth')
# 3) Llama al método model() para obtener la red, y pásala a modo eval
model = plan.model().to(device).eval()   # <- ojo a los paréntesis
model.eval()

NameError: name 'torch' is not defined

In [9]:

# 4) Ahora ya puedes inferir sobre tus datos
df = pd.read_csv('datos_limpios_prueba.csv') 
# 3) Convierte el DataFrame a tensor [N, features]
X = torch.tensor(df.values, dtype=torch.float32, device=device)
# 4) Inferencia en batch y vuelca al DataFrame
with torch.no_grad():
    logits = model(X)                 # shape [N, num_clases]
    preds  = logits.argmax(dim=1).cpu().numpy()

df['predicción'] = preds
df.to_csv('predicciones.csv', index=False)
print("Predicciones guardadas en predicciones.csv")


Predicciones guardadas en predicciones.csv
